In [22]:
import speech_recognition as sr
import os
#from vosk import Model
import seaborn as sns
import pandas as pd
import neattext



In [23]:
r = sr.Recognizer()
# # audio = sr.AudioFile("C:\\Users\\dawbr\\Desktop\\podftp\\midlands_english_male\\mim_02484_00159368346.wav")
# wi#th audio as source:
#     #r.adjust_for_ambient_noise(source, source)
#     audi#o_data = r.record(source)

In [24]:
#text = r.recognize_vosk(audio_data)#jhdashj
#print(text)

Transcribing all .wav files from directory

In [25]:
#import speech_recognition as sr
# import osdssdsa

#initialize recognizer
#r = sr.Recognizer()

#target directory containing all .wav files
#directory = os.fsencode("C:\\Users\\dawbr\\Desktop\\podftp\\midlands_english_male") #lokalizacja jest na bitach dlatego funkcja fsencode

# #loop trough all files in dir    
# #for file in os.listdir(directory):
#     #filename = os.fsdecode(file) #funkcja uzywa nazwy pliku wiec uzyje fsdecode zwracajacy str
#     #check if file is .wav
#     if filename.endswith(".wav"):
#         #load audio file to memory
#         audio = sr.AudioFile("C:\\Users\\dawbr\\Desktop\\podftp\\midlands_english_male\\" + filename)
#         with audio as source:
#             #read audio file
#             audio_data = r.record(source)
#         #transcribe audio file using vosk library (needs model folder avaialbe at https://alphacephei.com/vosk/models)
#         text = r.recognize_vosk(audio_data)
#         print(text)

Find ur microphone

In [26]:
# # import speech_recognition as sr
# for index, name in enumerate(sr.Microphone.list_microphone_names()):
#     print("Microphone with name \"{1}\" found for `Microphone(device_index={0})`".format(index, name))# # 

In [27]:
microphone = sr.Microphone(device_index=15)

In [28]:
text = ""
with microphone as source:
    print("Say something!")
    try:
        audio = r.listen(source, timeout=3)
        print("listening finished")
        # Convert audio to text
        text = r.recognize_vosk(audio)
        print("You said:", text)
    except sr.WaitTimeoutError:
        print("No speech detected after 3 seconds.")

Say something!
listening finished
You said: {
  "text" : "i have a bad day"
}


In [29]:
#analizujemy zmienna text

In [30]:
#slowa kluczowe jesli cos z listy jest w tekscie to bedzie slowem kluczowym, chodzi o odmiiany

word_list = ["kot", "pies", "ryba", "ptak"]


found = any(word in text for word in word_list)

if found:
    print('11')
else:
    print('0')

0


In [31]:
#wrzucac w dataframe zdanie w ktorym bylo dane slowo klucz typu "kot" a nastepnie dopisanie czasu gdy to powiedziano i sentyment tego zdania na koncu programu

In [32]:
from textblob import TextBlob
blob = TextBlob(text)
sentiment = blob.sentiment.polarity
if sentiment>0:
    print('sentence is positive')
elif sentiment<0:
    print('sentence is negative')
else:
    print('sentence is neutral')



sentence is negative


In [33]:
import pandas as pd
from datetime import datetime

# Sprawdź, czy plik CSV już istnieje

df = pd.read_csv('kek.csv')


current_datetime = datetime.now()

# Utwórz nowy DataFrame dla nowego wiersza
new_row = pd.DataFrame([[text, current_datetime.date(), current_datetime.time(), sentiment]], columns=['Text', 'Date', 'Time', 'Sentiment'])

# Połącz nowy DataFrame z istniejącym DataFrame
df = pd.concat([df, new_row], ignore_index=True)

# Zapisz DataFrame do pliku CSV
df.to_csv('data.csv', index=False)

# Wyświetl DataFrame
df

C:\Users\AdrianKurzac(261404)\AppData\Local\Temp\ipykernel_17296\1564249215.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


,Text,Date,Time,Sentiment
0,"{\n ""text"" : ""i have a bad day""\n}",2024-02-21,19:36:15.737176,-0.7


In [34]:
import neattext.functions as nfx

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [36]:
df1 = pd.read_csv('emotion_dataset_2.csv')
df1.head()
df1 = df1.dropna()

In [37]:
X = df1['Clean_Text']
Y = df1['Emotion']

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

In [39]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
pipe_LR = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(max_iter=2000,solver='lbfgs',C =2))])
pipe_RF = Pipeline(steps=[('cv',CountVectorizer()),('lr',RandomForestClassifier())])
pipe_SVC = Pipeline(steps=[('cv',CountVectorizer()),('lr',SVC())])
pipe_GB = Pipeline(steps=[('cv',CountVectorizer()),('lr',xgb.XGBClassifier(objective ='multi:softmax',  
    num_class= len(set(Y_train)),  
    max_depth= 3,  
    eta= 0.1,  
    subsample= 0.8,  
    colsample_bytree= 0.8,))])
pipe_GBR = Pipeline(steps=[('cv',CountVectorizer()),('lr',GradientBoostingRegressor(learning_rate=0.2,max_depth=8,n_estimators=200))])


In [40]:
# pipe_RF.fit(X_train,Y_train)
# pipe_RF.score(X_test,Y_test)# dokladnosc 0.59

In [41]:
# pipe_SVC.fit(X_train,Y_train)
# pipe_SVC.score(X_test,Y_test)#0.61

In [42]:
# from sklearn.preprocessing import LabelEncoder
# Y_train_encoded = LabelEncoder().fit_transform(Y_train)
# Y_test_encoded = LabelEncoder().fit_transform(Y_test)
# pipe_GB.fit(X_train,Y_train_encoded)
# pipe_GB.score(X_test,Y_test_encoded)#0.52

In [43]:
# pipe_GBR.fit(X_train,Y_train_encoded)
# pipe_GBR.score(X_test,Y_test_encoded)# 0.30

In [44]:
# pipe_SVC = Pipeline(steps=[('cv',CountVectorizer()),('lr',SVC(kernel = 'rbf'))])
# pipe_SVC.fit(X_train,Y_train)
# pipe_SVC.score(X_test,Y_test)

In [45]:
# pipe_SVC = Pipeline(steps=[('cv',CountVectorizer()),('lr',SVC(kernel = 'linear'))])
# pipe_SVC.fit(X_train,Y_train)
# pipe_SVC.score(X_test,Y_test)

In [46]:
# pipe_SVC = Pipeline(steps=[('cv',CountVectorizer()),('lr',SVC(kernel = 'poly'))])
# pipe_SVC.fit(X_train,Y_train)
# pipe_SVC.score(X_test,Y_test)

In [61]:
pipe_LR.fit(X_train,Y_train)
# pipe_LR.score(X_test,Y_test)


Pipeline(steps=[('cv', CountVectorizer()),
                ('lr', LogisticRegression(C=2, max_iter=2000))])

In [66]:
# pipe_LR.predict(['i am so happy to code'])
# text = text.split(':')[0]
z = pipe_LR.predict([str(text)])
print(text, z)
if sentiment>0:
    print('sentence is positive')
elif sentiment<0:
    print('sentence is negative')
else:
    print('sentence is neutral')


 "i have a bad day"
} ['sadness']
sentence is negative
